In [6]:
# dependencies
"""
Numpy: matrix manipulation and math
Pandas: csv parsing and various data structure tasks
Mathpltlib.pyplot: data visualization
set_trace: debug breaks
keras: a machine learning library that is intuitive to read
tensorflow: backend for keras... also the most widely used machine learning library
re: regular expressions
"""
from copy import deepcopy as copy
from IPython.core.debugger import set_trace

import numpy as np
import pandas as pd
import scipy.special as sci
import matplotlib.pyplot as plt 
import os
import tensorflow as tf
import keras
# not needed
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation

tf.config.optimizer.set_jit(True) # optimizes cpu usage

import re
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline

In [2]:
"""
concat_files
----------

Concatenate text files in a directory as a string

dependent on 'os' Python module

parameters
----------
directory: string; path of the target directory

f_type:    tuple of strings; target file extentsions Ex: ('.py', '.cpp')

return
---------
content:   concatenated string

"""
def concat_files(directory,f_type):
    import os
    # List all file in the dataset directory
    # ------------------
    all_file = []
    content = ""

    # walk through every directory and open every f_type file
    # concatenate into var string "content"
    for root, dirs, files in os.walk(directory): 
        for name in files:
            if name.endswith(f_type): # we only care about .py
                all_file.append(name)
                with open(os.path.join(root,name), "r") as f:
                    content += f.read() + "\n"
    return content

In [3]:
content = concat_files("dataset",('.py'))

In [4]:
r_all_ascii = "[^\x00-\x7F]"

In [15]:
"""
encode_string
-----------
Generate a dictionary representation of the characters found 
in a string keyed with integer representations

Returns two dictionaries and an array. The two dictionaries are 
necessary to convert the string to integer representation
and back again. The array is the string encoded as integer values.

parameters
----------
content:      string; to be processed

return
----------
vocab_to_int: dict; character to integer representation of unique characters in the string

int_to_vocab: dict; integer to string representation

encoded:      array; string encoded as integer values
"""

def encode_string(content):   
    # Convert the string "content" into a list of intergers
    
    ### creates a set of the individual characters
    vocab = set(content)
    ### attempt to clean out non-ascii characters
    vocab_c = copy(vocab)
    for i, char in enumerate(vocab_c):
        if re.search(r_all_ascii,char):
            vocab.remove(char)
    print(vocab)
    print(len(vocab))
    ### use the set to sequentially generate a dictionary
    vocab_to_int = {c: i for i, c in enumerate(vocab)} 
    print(vocab_to_int)
    ### make keys the numerical values
    int_to_vocab = dict(enumerate(vocab)) 
    
    ### encode the "content" string using dict
    ### encoded = np.array([vocab_to_int[c] for c in content], dtype=np.int32)
    
    # *** Uncomment the below lines if you haven't saved the encoded array
    # Then rerun cell
#   -------------------------------------------------
#     encoded = np.array([],dtype=np.int16)
#     for c in content:
#         if c in vocab_to_int:
#             encoded = np.append(encoded,vocab_to_int[c])
#   -------------------------------------------------
    encoded = np.load('./encoded.npy') # comment out if above lines are uncommented
    
    return vocab_to_int, int_to_vocab, encoded

In [16]:
vocab_to_int, int_to_vocab, encoded = encode_string(content)

{'[', 'i', 'Y', 'V', '$', 'H', 't', 'm', '~', '6', 'a', 'd', 'Q', 'M', '\t', '%', 'y', '_', '2', ']', 'Z', '{', '|', 'b', 'v', '.', 'w', '7', 'D', ',', '\\', '/', '`', 'j', '*', 'e', 'l', 'n', 'S', ';', 'B', 'E', '<', 'g', 'f', 'L', '8', '+', '#', '&', ' ', 'G', 'C', '"', 'c', ')', '\n', 'h', '5', '!', '^', 'K', '9', 'x', ':', 'o', 'k', '3', 'z', 'N', 'J', 'I', 'U', '-', '>', '=', 'A', '?', 'W', 'p', "'", '@', 'r', 'X', 'u', '4', 'F', 'P', 'T', '(', 'R', 's', 'O', '0', '}', 'q', '1'}
97
{'[': 0, 'i': 1, 'Y': 2, 'V': 3, '$': 4, 'H': 5, 't': 6, 'm': 7, '~': 8, '6': 9, 'a': 10, 'd': 11, 'Q': 12, 'M': 13, '\t': 14, '%': 15, 'y': 16, '_': 17, '2': 18, ']': 19, 'Z': 20, '{': 21, '|': 22, 'b': 23, 'v': 24, '.': 25, 'w': 26, '7': 27, 'D': 28, ',': 29, '\\': 30, '/': 31, '`': 32, 'j': 33, '*': 34, 'e': 35, 'l': 36, 'n': 37, 'S': 38, ';': 39, 'B': 40, 'E': 41, '<': 42, 'g': 43, 'f': 44, 'L': 45, '8': 46, '+': 47, '#': 48, '&': 49, ' ': 50, 'G': 51, 'C': 52, '"': 53, 'c': 54, ')': 55, '\n': 56, '

## $\rightarrow$ Save encoded array to avoid heavy computation

In [12]:
#from tempfile import TemporaryFile as TF
outfile = "./encoded"

np.save(outfile,encoded)

In [17]:
#print(content)
print(int_to_vocab)
# this is all of the files concatenated. with each character encoded using the int_to_vocab
print(encoded)

{0: '[', 1: 'i', 2: 'Y', 3: 'V', 4: '$', 5: 'H', 6: 't', 7: 'm', 8: '~', 9: '6', 10: 'a', 11: 'd', 12: 'Q', 13: 'M', 14: '\t', 15: '%', 16: 'y', 17: '_', 18: '2', 19: ']', 20: 'Z', 21: '{', 22: '|', 23: 'b', 24: 'v', 25: '.', 26: 'w', 27: '7', 28: 'D', 29: ',', 30: '\\', 31: '/', 32: '`', 33: 'j', 34: '*', 35: 'e', 36: 'l', 37: 'n', 38: 'S', 39: ';', 40: 'B', 41: 'E', 42: '<', 43: 'g', 44: 'f', 45: 'L', 46: '8', 47: '+', 48: '#', 49: '&', 50: ' ', 51: 'G', 52: 'C', 53: '"', 54: 'c', 55: ')', 56: '\n', 57: 'h', 58: '5', 59: '!', 60: '^', 61: 'K', 62: '9', 63: 'x', 64: ':', 65: 'o', 66: 'k', 67: '3', 68: 'z', 69: 'N', 70: 'J', 71: 'I', 72: 'U', 73: '-', 74: '>', 75: '=', 76: 'A', 77: '?', 78: 'W', 79: 'p', 80: "'", 81: '@', 82: 'r', 83: 'X', 84: 'u', 85: '4', 86: 'F', 87: 'P', 88: 'T', 89: '(', 90: 'R', 91: 's', 92: 'O', 93: '0', 94: '}', 95: 'q', 96: '1'}
[48 59 31 ... 55 56 56]


In [31]:
segment = [50,100,150,200]
# for i in range(500): 
#     segment.append((int(len(encoded) / 500) * (i + 1)))
batch = {
    "x" : [
        encoded[:segment[0]],
        encoded[segment[0]:segment[1]]
    ],
    "y" : [
        encoded[segment[1]:segment[2]],
        encoded[segment[2]:segment[3]]
    ]
}

In [32]:
# inp = np.vstack((batch['x'][0],batch['x'][1]))
# targets = np.vstack((batch['y'][0],batch['y'][1]))
inp = batch['x'][0]
targets = batch['y'][0]
Xtest = batch['x'][1]
Ttest = batch['y'][1]
io_size = len(int_to_vocab)

# inp = np.hstack((np.ones((Xtest.shape[0],1)),tf.one_hot(inp,io_size)))
# targets = np.hstack((np.ones((Xtest.shape[0],1)),tf.one_hot(targets,io_size)))
# Xtest = np.hstack((np.ones((Xtest.shape[0],1)),tf.one_hot(Xtest,io_size)))
# Ttest = np.hstack((np.ones((Xtest.shape[0],1)),tf.one_hot(Ttest,io_size)))

inp = tf.one_hot(inp,io_size)
targets = tf.one_hot(targets,io_size)
Xtest = tf.one_hot(Xtest,io_size)
Ttest = tf.one_hot(Ttest,io_size)

# encoded_as_i = np.hstack((np.ones((1,1)),tf.one_hot(encoded[:1],io_size)))
encoded_as_i = tf.one_hot(encoded[:40],io_size)

#tf.print(Xtest, summarize=50)
print(inp.shape)

# inp = tf.expand_dims(inp,2)
# targets = tf.expand_dims(targets,2)

(50, 97)


In [109]:
# lines = []
# temp = []
# for i, char in enumerate(encoded):
#     if int_to_vocab[char] == '\n':
#         lines.append(temp)
#     else:
#         temp.append(char)
#         temp = []

# x = np.zeros((len(lines), maxlen, len(int_to_vocab)), dtype=np.bool)
# y = np.zeros((len(sentences), len(int_to_vocab)), dtype=np.bool)
# for i, sentence in enumerate(sentences):
#     for t, char in enumerate(sentence):
#         x[i, t, char_indices[char]] = 1
#     y[i, char_indices[next_chars[i]]] = 1

In [43]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import LSTM
from keras.optimizers import RMSprop, SGD

# x_train = np.hstack((batch['x'][0],batch['x'][1]))
# y_train = np.hstack((batch['y'][0],batch['y'][1]))
# x_test = np.hstack((batch['x'][0],batch['x'][1]))
# y_test = np.hstack((batch['y'][0],batch['y'][1]))

max_features = 97
#maxlen = 40

model = Sequential()
model.add(Embedding(max_features, output_dim=97))
model.add(LSTM(97, input_shape=inp.shape))
model.add(Dropout(0.5))
model.add(Dense(io_size, activation='softmax'))

optimizer = RMSprop(learning_rate=0.01)

model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

model.fit(Xtest, Xtest, steps_per_epoch=50, epochs=5)
#model.get_weights().shape

/home/bran/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/5
50/50 [==============================] - 18s 363ms/step - loss: 3.0160 - accuracy: 0.1752
Epoch 2/5
50/50 [==============================] - 16s 323ms/step - loss: 2.8579 - accuracy: 0.2060
Epoch 3/5
50/50 [==============================] - 17s 341ms/step - loss: 2.2597 - accuracy: 0.2884
Epoch 4/5
50/50 [==============================] - 15s 308ms/step - loss: 1.8770 - accuracy: 0.4268
Epoch 5/5
50/50 [==============================] - 16s 316ms/step - loss: 1.5137 - accuracy: 0.4972


In [44]:
from keras.models import load_model
model.save('./model')
score = model.evaluate(inp, inp, steps=50)
print(score)

50/50 [==============================] - 4s 79ms/step
[0.12511909484863282, 23.000000417232513]


In [64]:
# print(tf.keras.backend.shape(Xtest),tf.keras.backend.shape(model.weights))
# sci.softmax(Xtest @ encoded_to_i)
print(encoded_as_i)
print(tf.convert_to_tensor(model.predict(encoded_as_i, steps=3, verbose=0)))
print(np.argmax(model.predict(encoded_as_i, steps=3, verbose=0),axis=0))
print(np.sum(model.predict(encoded_as_i, steps=3, verbose=0)[0]))
nextc = int_to_vocab[np.argmax(model.predict(encoded_as_i, steps=3, verbose=0),axis=0)[1]]
print(nextc) # should give probabilities of next character
print()

tf.Tensor(
[[0. 0. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]], shape=(40, 97), dtype=float32)
tf.Tensor(
[[4.0382799e-02 3.6107301e-07 2.2326464e-07 ... 2.5392863e-05
  2.9392481e-07 2.9682593e-07]
 [3.0115703e-02 9.5992255e-07 5.8833479e-07 ... 3.8130096e-05
  7.7778634e-07 7.9772832e-07]
 [4.4122519e-04 1.1850213e-07 7.9293002e-08 ... 4.3012276e-02
  1.0135348e-07 1.2478394e-07]
 ...
 [1.2823937e-03 1.7435547e-06 1.2205473e-06 ... 4.7490899e-05
  1.3142378e-06 1.4809921e-06]
 [1.5982764e-02 4.5820761e-06 2.8151956e-06 ... 7.4550655e-05
  3.8831436e-06 4.0619348e-06]
 [3.8662888e-02 4.1631239e-07 2.5747269e-07 ... 2.6746873e-05
  3.3779594e-07 3.4244664e-07]], shape=(120, 97), dtype=float32)
[ 0  8  8 34  8  8  0  0 34  8  8  8 34 34 34  8 34  8 34 34  0  0  0  1
  1 34  8  8  8  9  8  8 34  8  8  8  8  8  8  8  8 34  8 34 34  8 34  8
 34  8 34 13 34  8 11 34  8  8 34  8  8 34  8  

In [47]:
strout = ""
data = Xtest
#actual = ""
predict = np.argmax(model.predict(data, steps=70, verbose=0),axis=0)
for i in predict:
    
    nextc = int_to_vocab[predict[i]]
    
    #actual += int_to_vocab[data[i]]
    strout += nextc
print(strout)
#print(actual)


iiiiiiiiiittiiiiyiiiiiiiiiyiiiiiiitittiittiiiiiiiitiiiiitiiiiiii\\\iiiiiiiiiiiiiiititiitiiitiiiii


# References

1. [LSTM: A search space odyssey](https://arxiv.org/pdf/1503.04069.pdf?utm_content=buffereddc5&utm_medium=social&utm_source=plus.google.com&utm_campaign=buffer)